# Encuesta Servicios

In [ ]:
# Cargar paquetes
from datetime import datetime
import pandas as pd
from sqlalchemy import and_, or_
from sqlalchemy.sql import func
from lib.database import SessionLocal
db = SessionLocal()
from tabulate import tabulate

In [ ]:
# Cargar modelos
from citas_admin.v2.autoridades.models import Autoridad
from citas_admin.v2.cit_categorias.models import CitCategoria
from citas_admin.v2.cit_citas.models import CitCita
from citas_admin.v2.cit_clientes.models import CitCliente
from citas_admin.v2.cit_clientes_recuperaciones.models import CitClienteRecuperacion
from citas_admin.v2.cit_clientes_registros.models import CitClienteRegistro
from citas_admin.v2.cit_dias_inhabiles.models import CitDiaInhabil
from citas_admin.v2.cit_horas_bloqueadas.models import CitHoraBloqueada
from citas_admin.v2.cit_oficinas_servicios.models import CitOficinaServicio
from citas_admin.v2.cit_servicios.models import CitServicio
from citas_admin.v2.distritos.models import Distrito
from citas_admin.v2.domicilios.models import Domicilio
from citas_admin.v2.enc_servicios.models import EncServicio
from citas_admin.v2.enc_sistemas.models import EncSistema
from citas_admin.v2.materias.models import Materia
from citas_admin.v2.modulos.models import Modulo
from citas_admin.v2.oficinas.models import Oficina
from citas_admin.v2.permisos.models import Permiso
from citas_admin.v2.roles.models import Rol
from citas_admin.v2.usuarios.models import Usuario
from citas_admin.v2.usuarios_roles.models import UsuarioRol

In [ ]:
enc_servicios = db.query(EncServicio.id, EncServicio.estado, EncServicio.oficina_id).limit(10)
print(enc_servicios.all())

In [ ]:
enc_servicios = db.query(
    EncServicio.estado.label("estado"),
    func.count("*").label("cantidad"),
).group_by(EncServicio.estado)
for estado, cantidad in enc_servicios.all():
    print(f"Estado {estado} tiene {cantidad}")

In [ ]:
conteo_respuesta_01 = db.\
    query(
        EncServicio.respuesta_01.label("respuesta_01"),
        func.count("*").label("cantidad"),
    ).\
    filter(EncServicio.respuesta_01 != None).\
    group_by(EncServicio.respuesta_01)

tabla = tabulate(conteo_respuesta_01.all(), ["respuesta_01", "cantidad"])
print(tabla)

In [ ]:
subquery1 = db.\
    query(
        func.count(EncServicio.respuesta_01).filter(EncServicio.respuesta_01 == 1).label("muy_dificil"),
        func.count(EncServicio.respuesta_01).filter(EncServicio.respuesta_01 == 2).label("dificil"),
        func.count(EncServicio.respuesta_01).filter(EncServicio.respuesta_01 == 3).label("neutro"),
        func.count(EncServicio.respuesta_01).filter(EncServicio.respuesta_01 == 4).label("facil"),
        func.count(EncServicio.respuesta_01).filter(EncServicio.respuesta_01 == 5).label("muy_facil"),
    ).subquery()

subquery2 = db.\
    query(
        func.count(EncServicio.respuesta_02).filter(EncServicio.respuesta_02 == 1).label("muy_dificil"),
        func.count(EncServicio.respuesta_02).filter(EncServicio.respuesta_02 == 2).label("dificil"),
        func.count(EncServicio.respuesta_02).filter(EncServicio.respuesta_02 == 3).label("neutro"),
        func.count(EncServicio.respuesta_02).filter(EncServicio.respuesta_02 == 4).label("facil"),
        func.count(EncServicio.respuesta_02).filter(EncServicio.respuesta_02 == 5).label("muy_facil"),
    ).subquery()

todo = db.query(subquery1, subquery2)
print(tabulate(todo.all()))

In [ ]:
import pandas as pd

cant_01 = db.\
    query(EncServicio.respuesta_01.label("respuesta"), func.count("*").label("cantidad")).\
    filter(EncServicio.respuesta_01 != None).\
    group_by(EncServicio.respuesta_01)
cant_01_df = pd.DataFrame(cant_01.all())
cant_01_df["opcion"] = "respuesta_01"

cant_02 = db.\
    query(EncServicio.respuesta_02.label("respuesta"), func.count("*").label("cantidad")).\
    filter(EncServicio.respuesta_02 != None).\
    group_by(EncServicio.respuesta_02)
cant_02_df = pd.DataFrame(cant_02.all())
cant_02_df["opcion"] = "respuesta_02"

cant_03 = db.\
    query(EncServicio.respuesta_03.label("respuesta"), func.count("*").label("cantidad")).\
    filter(EncServicio.respuesta_03 != None).\
    group_by(EncServicio.respuesta_03)
cant_03_df = pd.DataFrame(cant_03.all())
cant_03_df["opcion"] = "respuesta_03"

todo = pd.concat([cant_01_df, cant_02_df, cant_03_df])

pivot_table = todo.pivot_table(index="respuesta", columns="opcion")
print(pivot_table)